In [1]:
import cv2
import numpy as np

In [2]:
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

In [3]:
classes = []
with open('coco','r') as f:
    classes = f.read().splitlines()

In [4]:
img = cv2.imread('4.jpeg')
height, width , _ = img.shape

In [5]:
blob = cv2.dnn.blobFromImage(img , 1/255 , (416,416) , (0,0,0) , swapRB=True , crop=False)
net.setInput(blob)

In [6]:
output_layers_names = net.getUnconnectedOutLayersNames()
layersOutputs = net.forward(output_layers_names)

In [7]:
boxes = []
confidences = []
class_ids = []
for output in layersOutputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0]*width)
            center_y = int(detection[1]*height)
            w  = int(detection[2]*width)
            h  = int(detection[3]*height)
            
            x  = int(center_x - w/2)
            y  = int(center_y - h/2)
            
            boxes.append([x , y , w, h])
            confidences.append((float(confidence)))
            class_ids.append(class_id)

In [8]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

In [9]:
font = cv2.FONT_HERSHEY_PLAIN
colours = np.random.uniform(0,255,size=((len(boxes),3)))
for i in indexes.flatten():
    x,y,w,h = boxes[i]
    label = str(classes[class_ids[i]])
    confidence = str(round(confidences[i],2))
    colour = colours[i]
    cv2.rectangle(img,(x,y),(x+w , y+h), colour , 2)
    cv2.putText(img , label + " "+ confidence , (x , y+20) , font , 2 , (255,255,255) , 2)

In [10]:
cv2.imshow('Image',img)
cv2.waitKey(0)

-1